In [ ]:
!pip install opencv-python-headless

In [ ]:
!pip install pytesseract

In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
import pytesseract

In [ ]:
class HsvFilter:
    def __init__(self, hMin=None, sMin=None, vMin=None, hMax=None, sMax=None, vMax=None, sAdd=None, sSub=None, vAdd=None, vSub=None):
        self.hMin = hMin
        self.sMin = sMin
        self.vMin = vMin
        self.hMax = hMax
        self.sMax = sMax
        self.vMax = vMax
        self.sAdd = sAdd
        self.sSub = sSub
        self.vAdd = vAdd
        self.vSub = vSub

In [ ]:
def init_control_gui():
    cv.namedWindow('HSV Filters')
    cv.resizeWindow('HSV Filters', 700, 600)

    def nothing(x):
        pass

    # create trackbars for color change
    # Hue is from 0-179 for Opencv
    cv.createTrackbar('HMin', 'HSV Filters', 0, 179, nothing)
    cv.createTrackbar('SMin', 'HSV Filters', 0, 255, nothing)
    cv.createTrackbar('VMin', 'HSV Filters', 0, 255, nothing)
    cv.createTrackbar('HMax', 'HSV Filters', 0, 179, nothing)
    cv.createTrackbar('SMax', 'HSV Filters', 0, 255, nothing)
    cv.createTrackbar('VMax', 'HSV Filters', 0, 255, nothing)

    # Set default value for MAX HSV trackbars.
    cv.setTrackbarPos('HMax', 'HSV Filters', 179)
    cv.setTrackbarPos('SMax', 'HSV Filters', 255)
    cv.setTrackbarPos('VMax', 'HSV Filters', 255)

    # Create HSV switch
    cv.createTrackbar('SAdd', 'HSV Filters', 0, 255, nothing)
    cv.createTrackbar('SSub', 'HSV Filters', 0, 255, nothing)
    cv.createTrackbar('VAdd', 'HSV Filters', 0, 255, nothing)
    cv.createTrackbar('VSub', 'HSV Filters', 0, 255, nothing)
    
    # Text Filtering
    # hsv_filter.hMin = 0
    # hsv_filter.sMin = 0
    # hsv_filter.vMin = 0
    # hsv_filter.hMax = 176
    # hsv_filter.sMax = 255
    # hsv_filter.vMax = 255
    # hsv_filter.sAdd = 255
    # hsv_filter.sSub = 0
    # hsv_filter.vAdd = 69
    # hsv_filter.vSub = 71
    
    # Circle Filtering
    # hsv_filter.hMin = 0
    # hsv_filter.sMin = 58
    # hsv_filter.vMin = 0
    # hsv_filter.hMax = 115
    # hsv_filter.sMax = 255
    # hsv_filter.vMax = 255
    # hsv_filter.sAdd = 0
    # hsv_filter.sSub = 0
    # hsv_filter.vAdd = 0
    # hsv_filter.vSub = 0
    
    #test push
    
    return hsv_filter

In [ ]:
def apply_hsv_filter(original_image, hsv_filter=None):
    # convert image to HSV
    hsv = cv.cvtColor(original_image, cv.COLOR_BGR2HSV)

    # if we haven't been given a defined filter, use the filter values from the GUI
    if not hsv_filter:
        hsv_filter = get_hsv_filter_from_controls()

    # add/subtract saturation and value
    h, s, v = cv.split(hsv)
    s = shift_channel(s, hsv_filter.sAdd)
    s = shift_channel(s, -hsv_filter.sSub)
    v = shift_channel(v, hsv_filter.vAdd)
    v = shift_channel(v, -hsv_filter.vSub)
    hsv = cv.merge([h, s, v])

    # Set minimum and maximum HSV values to display
    lower = np.array([hsv_filter.hMin, hsv_filter.sMin, hsv_filter.vMin])
    upper = np.array([hsv_filter.hMax, hsv_filter.sMax, hsv_filter.vMax])
    # Apply the thresholds
    mask = cv.inRange(hsv, lower, upper)
    result = cv.bitwise_and(hsv, hsv, mask=mask)

    # convert back to BGR for imshow() to display it properly
    img = cv.cvtColor(result, cv.COLOR_HSV2BGR)

    return img


def shift_channel(c, amount):
    if amount > 0:
        lim = 255 - amount
        c[c >= lim] = 255
        c[c < lim] += amount
    elif amount < 0:
        amount = -amount
        lim = amount
        c[c <= lim] = 0
        c[c > lim] -= amount
    return c

In [ ]:
# Load the images
image_path = '../images/'
right_box = cv.imread(image_path/'right.png')
left_box = cv.imread(image_path/'left.png')
mandala = cv.imread(image_path/'manda')

In [ ]:
import cv2 as cv
import numpy as np
import pytesseract
from matplotlib import pyplot as plt

class HsvFilter:
    def __init__(self, hMin=None, sMin=None, vMin=None, hMax=None, sMax=None, vMax=None, sAdd=None, sSub=None, vAdd=None, vSub=None):
        self.hMin = hMin
        self.sMin = sMin
        self.vMin = vMin
        self.hMax = hMax
        self.sMax = sMax
        self.vMax = vMax
        self.sAdd = sAdd
        self.sSub = sSub
        self.vAdd = vAdd
        self.vSub = vSub

# circle filtering
hsv_filter = HsvFilter()
hsv_filter.hMin = 0
hsv_filter.sMin = 58
hsv_filter.vMin = 0
hsv_filter.hMax = 115
hsv_filter.sMax = 255
hsv_filter.vMax = 255
hsv_filter.sAdd = 0
hsv_filter.sSub = 0
hsv_filter.vAdd = 0
hsv_filter.vSub = 0

# Load the image
img = cv.imread('fullmandala.png')

# Convert the image to HSV
hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

# Define the HSV range for the color
lower_range = np.array([hsv_filter.hMin, hsv_filter.sMin, hsv_filter.vMin])
upper_range = np.array([hsv_filter.hMax, hsv_filter.sMax, hsv_filter.vMax])

# Create a mask based on the HSV range
mask = cv.inRange(hsv, lower_range, upper_range)

# Use the mask to extract the color from the image
res = cv.bitwise_and(img, img, mask=mask)

# Display the original image and the masked image
plt.subplot(121),plt.imshow(img),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(res),plt.title('Masked')
plt.xticks([]), plt.yticks([])
plt.show()